In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
path_q_measurements = r"c:\Users\NLHARN\Sweco AB\51024754_Vallei_en_Veluwe_Afwateringseenheden - General\\Aangeleverd door Waterschap\\Afvoeren Azure\\Q-afvoergebieden_Azure\\Q-afvoergebieden_Azure\\Q(CMS)_WSVV.csv"

In [ ]:
q = pd.read_csv(
    path_q_measurements, 
    sep=';', 
    skiprows=1, 
    index_col=0, 
    parse_dates=[0],
    dayfirst=True,
    na_values=[-999, -999.0, -9999, 'NA', 'nan', '']  # treat these as NaN on read
)

In [ ]:
q["SL000417"].plot(figsize=(10,5), title="Afvoermetingen SL000417")

In [ ]:
duurcurve = q["SL000417"].sort_values(ascending=False).dropna().reset_index(drop=True)

In [ ]:
duurcurve.plot()

In [ ]:
duurcurve.index = duurcurve.index / len(duurcurve) * 365  # convert to percentage

In [ ]:
duurcurve.plot()

In [ ]:
duurcurve.head(20)

In [ ]:
len(q["SL000417"].dropna())/365

In [ ]:
s = q["SL000417"].astype(float)

threshold = 45

try:
    from scipy.signal import find_peaks
    peaks_idx, props = find_peaks(s.values, height=threshold)
    peaks = s.iloc[peaks_idx]
except Exception:
    peaks = s[(s > threshold) & (s > s.shift(1)) & (s >= s.shift(-1))]

ax = s.plot(figsize=(10,5), title=f"Afvoermetingen SL000417 — peaks > {threshold:.2f}")
ax.scatter(peaks.index, peaks.values, color="red", zorder=5)
for x, y in zip(peaks.index, peaks.values):
    ax.annotate(f"{y:.1f}", xy=(x, y), xytext=(0, 6), textcoords="offset points", ha="center", fontsize=8)
print(f"Found {len(peaks)} peaks. Example:\n", peaks.head())